# Swarm Simulator for Outer Space

This swarm simulator will explore different swarm algorithms for an application in outer space. This approach will divide a plot into grid squares, which can then be analysed to determine and control the behaviour of each individual agent in the swarm.

In [42]:
#imports
import sympy as sym
import plotly as py
import plotly.tools as tls
from plotly import graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
from IPython.display import Image

import dash
from dash.dependencies import Output, Input
import dash_core_components as dcc
import dash_html_components as html

import numpy as np
import pandas as pd
import ipywidgets as widgets
import time
import math

from scipy import special
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from collections import deque
from scipy.spatial.distance import cdist, pdist, euclidean
from sympy import sin, cos, symbols, lambdify

from robot import Robot
from dust_devil import DustDevil
from live_functions_tracking import update_decay, initialise,random_position, grid_center,positions,bounce,magnitude,unit,division_check,physics_walk,dust_check,update_timestep,cluster_function,G_transition,dist,random_walk,dust,trajectory_dust,update_dust,load_positions,pre_initialise

import random
import datetime
import os
import math
import sys

import Processing_Functions_Tracking


import dash
from dash.dependencies import Output, Input
import dash_core_components as dcc
import dash_html_components as html

import json
import pickle

from ast import literal_eval

app = dash.Dash(__name__)

In [43]:
print("Kernel Check")

Kernel Check


In [44]:
#Plotly offline mode
init_notebook_mode(connected=True)

In [45]:
#function to calculate area coverage using grid metric
def grid_coverage(grid,positions,length):
    #creating the grid bins for area coverage
    x_bins = np.linspace(grid[0],grid[1],length+1)
    y_bins = np.linspace(grid[0],grid[1],length+1)
    
    x_positions = positions[0]
    y_positions = positions[1]
    #initialising empty coordinates array for use in the grid metric
    coordinates = []
    for i in range(len(x_positions)):
        #setting current position
        x_current = x_positions[i]
        y_current = y_positions[i]
        #searching for the x anf y coordinates using the numpy search sorted function combined with the previously defined bins
        x = np.searchsorted(x_bins, x_current)
        y = np.searchsorted(y_bins, y_current)
        if((0<=x<length+1) and (0<=y<length+1)):
            coordinates.append((x,y))
        #print(coordinates)


    rm_duplicates = list(set(coordinates)) #set is used to remove duplicates, as sets require distinct elements
    area_coverage = len(rm_duplicates)*0.01 #converting it to a percentage
    total = length*length
    return area_coverage

In [46]:
#Function to call physics based area coverage algorithm
def physics_walk_tracking(swarm,G,power,R,max_force,multiply,timestep):
    '''
    Updates the robots velocities based on a physicomimetics artificial gravity algorithm

            Parameters:
                   Swarm (list): a list of robot objects
                   G (float): the chosen G constant float value
                   power (float): the chosen power for the artificial gravity equation
                   R (float): the chosen R seperation float value
                   max_force (float): the maximum float limit on the force
                   multiply (float): the range multiplication float
            Returns:
                   None

    '''
    #setting initial constants
    force_x = 0
    force_y = 0 
    velocity_x = 0
    velocity_y = 0
    
    #retrieving x and y positions
    x,y = positions(swarm)
    
    #looping through swarm with each individual robot
    for i in range(len(swarm)):
        
        #selecting robots one by one
        robot = swarm[i]
        #print("Robot Position: ", [robot.x,robot.y])
        
        x_copy,y_copy = x.copy(),y.copy()
        
        #initialising the forces as 0
        force = np.array([0,0])

        #finding the distance between the current robot and the other robots
        x_dist_unsorted = np.array(x_copy)-robot.x
        y_dist_unsorted = np.array(y_copy)-robot.y
        
        #calculating the distance from the robots to the current robot
        distance = np.sqrt(np.square(x_dist_unsorted)+np.square(y_dist_unsorted))

        #print("Distance: ",distance)
        #print("Non Zero Distance:", np.nonzero(distance))
        #determining the robots within 1.5R distance
        distance_local = np.logical_and(distance>0,distance<multiply*R)
        neighbourhood = np.where(distance_local)
        list_neighbours = list(neighbourhood[0])
        #print("Indices of local: ",list_neighbours)
        #print(len(np.where(distance_local)))
        #print("Booleans Local: ",distance_local)
        #print(type(distance_local))
        #creating the x and y distance matrices for robots within the local 1.5R distance
        neighbours = [swarm[i] for i in list_neighbours]
        #print("Neighbours:",positions(neighbours))
        x_dist = x_dist_unsorted[distance_local]
        y_dist = y_dist_unsorted[distance_local]
        position = np.array([x_dist,y_dist])
        
        #initialising force changes
        force_change_dir = 0
        
        #number of robots within the neighbourhood
        robot_number = len(distance_local)
        

        #looping through the current robots neighbourhood
        for j in range(len(neighbours)):
            
            neighbour = neighbours[j]
            
            #storing current position 
            current_position = np.array([position[0,j],position[1,j]])
                

            #calculating magnitude of positions
            mag = magnitude(current_position[0],current_position[1])


            #calculating the force
            numerator = (G*(robot.mass*neighbour.mass))
            denominator = mag**2


            #calling function to account for division by zero, if there is a zero denominator, then the force component with zero is returned as zero
            force_change = division_check(numerator,denominator)

            #calculating the unit vector of the position
            distance_unit = unit(current_position)

            #if the magnitude is bigger than R, then a force is added to draw the robots together             
            if(mag>robot.R):
                force_change_dir = distance_unit

            #if the magnitude is smaller than R, then a force is added to push the robots apart 
            elif(mag<robot.R):
                force_change_dir = -distance_unit
            else:
                force_change_dir = np.array([0,0])

            #calculating new force change based on direction and magnitude
            force_delta = force_change_dir*force_change

            #constraining force to the maximum
            if((force_delta[0]**2+force_delta[1]**2)>((max_force)**2)):
                #calculating unit vector of the force
                unit_force = unit([force_delta[0],force_delta[1]])
                
                #multiplying it by the maximum force
                updating_force = unit_force*max_force
                
                #setting the new force equal to the respective updated maximum force components
                force_delta[0] = updating_force[0]
                force_delta[1] = updating_force[1]

            #calculating new force
            force = force+force_delta
            

        #calculating the change in velocity
        delta_vx = (force[0])*timestep/robot.mass
        delta_vy = (force[1])*timestep/robot.mass

        #calculating the new velocity
        velocity_x = robot.x_velocity + delta_vx
        velocity_y = robot.y_velocity + delta_vy

       #keeping the velocity within the maximum velocity of the robot
        if((velocity_x**2+velocity_y**2)>((robot.max_velocity)**2)):
            #calculating unit vector of the velocity
            unit_velocity = unit([velocity_x,velocity_y])
            
            #multiplying it by the maximum velocity
            velocity = unit_velocity*robot.max_velocity
            
            #setting the new velocity equal to the respective updated maximum velocity components
            velocity_x = velocity[0]
            velocity_y = velocity[1]

        #updating the robots velocity
        robot.update_velocity(velocity_x,velocity_y)    


In [47]:

#Function to call physics based area coverage algorithm
def physics_walk_honey(swarm,G,power,R,max_force,multiply,timestep,lattice_constants,velocity_coefficient):
    '''
    Updates the robot position based on a physicomimetics algorithm

            Parameters:
                   Swarm (list): a list of robot objects
            
            Returns:
                   None

    '''

    #print("Physics Called \n===============")
    #setting initial constants
    #setting initial constants
    force_x = 0
    force_y = 0 
    velocity_x = 0
    velocity_y = 0
    x,y = np.array(positions(swarm))
    type_0,type_1 = split(swarm)
    x_0,y_0 = np.array(positions(type_0))
    x_1,y_1 = np.array(positions(type_1))
    #print("Length 0:",len(x_0))
    #print("Length 1: ",len(x_1))
    
    #looping through each robot in the swarm
    for i in range(len(swarm)):
        force_0 = np.array([0,0])
        force_1 = np.array([0,0])
        #selecting robots one by one
        robot = swarm[i]
        
        robot.update_velocity(velocity_coefficient*velocity_x,velocity_coefficient*velocity_y)    
        #initialising the forces as 0
        force = np.array([0,0])
        
        R_0 = R
        R_1 = R
        #x_dist_unsorted,y_dist_unsorted = dist(robot,swarm)
        
        #finding the distance between the current robot and the other robots
        delta_x0,delta_y0 = dist(robot,x_0,y_0)
        #print([delta_x0,delta_y0])
        delta_x1,delta_y1 = dist(robot,x_1,y_1)
        """#finding the distance between the current robot and the other robots
        delta_x0,delta_y0 = dist(robot,x,y)
        delta_x1,delta_y1 = dist(robot,x,y)"""
        
        #calculating the distance from the robots to the current robot, split up according to robot type
        distance_0 = np.sqrt(np.square(delta_x0)+np.square(delta_y0))
        distance_1 = np.sqrt(np.square(delta_x1)+np.square(delta_y1))
        
        #finding the corresponding booleans
        multiply_like = 1.3
        multiply_unlike = 1.7
        robot.update_counter()
        constant = 1/robot.lattice
        if(robot.identifier == 0):
            R_0 = R
            R_1 = R
            distance_local_0 = np.logical_and(distance_0>0,(distance_0*constant)<multiply_like*R)
            distance_local_1 = np.logical_and(distance_1>0,distance_1<multiply_unlike*R)
            like_0 = True
            like_1 = False
            neighbourhood_0 = np.where(distance_local_0)
            #print("Raw Indices: ", neighbourhood_0)
            list_neighbours_0 = list(neighbourhood_0[0])
            neighbours_0 = [type_0[k] for k in list_neighbours_0]
            #print(list_neighbours_0)
            
            
            neighbourhood_1 = np.where(distance_local_1)
            #print("Raw Indices: ", neighbourhood_1)
            list_neighbours_1 = list(neighbourhood_1[0])
            neighbours_1 = [type_1[l] for l in list_neighbours_1]
            #print(list_neighbours_1)
            
        if(robot.identifier == 1):
            like_0 = False
            like_1 = True
            R_0 = R
            R_1 = R
            distance_local_0 = np.logical_and(distance_0>0,distance_0<multiply_unlike*R)
            distance_local_1 = np.logical_and(distance_1>0,(distance_1*constant)<multiply_like*R)
    
            neighbourhood_0 = np.where(distance_local_0)
            #print("Raw Indices: ", neighbourhood_0)
            list_neighbours_0 = list(neighbourhood_0[0])
            neighbours_0 = [type_0[m] for m in list_neighbours_0]
            #print(list_neighbours_0)
            
            neighbourhood_1 = np.where(distance_local_1)
            #print("Raw Indices: ", neighbourhood_1)
            list_neighbours_1 = list(neighbourhood_1[0])
            neighbours_1 = [type_1[n] for n in list_neighbours_1]
            #print(list_neighbours_1)
            
        #print(like_0)
        #print(like_1)
        
        x_dist_0 = delta_x0[distance_local_0]
        y_dist_0 = delta_y0[distance_local_0]    
        positions_0 = np.array([x_dist_0,y_dist_0])
        
        x_dist_1 = delta_x1[distance_local_1]
        y_dist_1 = delta_y1[distance_local_1]
        positions_1 = np.array([x_dist_1,y_dist_1])
        
        #print("Neighbourhood Distances 0: ", positions_0)
        #print("Neighbourhood Distances 1: ", positions_1)
        #print("Neighbourhood Distance Magnitude 0: ", np.sqrt(np.square(positions_0[0])+np.square(positions_0[1])))
        #print("Neighbourhood Distance Magnitude 1: ", np.sqrt(np.square(positions_1[0])+np.square(positions_1[1])))
        force_0 = artificial_gravity(neighbours_0,positions_0,robot,max_force,R_0,G,like_0)
        force_1 = artificial_gravity(neighbours_1,positions_1,robot,max_force,R_1,G,like_1)
        
        friction = random.uniform(-0.1,0.1)
        force = force_0 + force_1 #+ friction

        #calculating the change in velocity
        delta_vx = force[0]*timestep/robot.mass
        delta_vy = force[1]*timestep/robot.mass

        #calculating the new velocity
        velocity_x = robot.x_velocity + delta_vx#velocity_coefficient*
        velocity_y = robot.y_velocity + delta_vy#velocity_coefficient*        
    
        #keeping the velocity within the maximum velocity of the robot
        if((velocity_x**2+velocity_y**2)>((robot.max_velocity)**2)):
            #calculating unit vector of the velocity
            unit_velocity = unit([velocity_x,velocity_y])
            
            #multiplying it by the maximum velocity
            velocity = unit_velocity*robot.max_velocity
            
            #setting the new velocity equal to the respective updated maximum velocity components
            velocity_x = velocity[0]
            velocity_y = velocity[1]
        
        #updating the robots velocity
        robot.update_velocity(velocity_x,velocity_y)    

        #calculating the change in position for x and y
        x_change = velocity_x*timestep
        y_change = velocity_y*timestep

        #calculating the new position
        x_updated = robot.x+x_change
        y_updated = robot.y+y_change
        
        #updating the position
        robot.update_position(x_updated,y_updated)
        #print("Finished 1 Loop within Physics Walk")
    #print("Finished the overall Physics Walk Calculation")

In [48]:
#Function to calculate artificial gravity forces
def artificial_gravity(neighbourhood,positions, robot, max_force,R,G,like):
        # print("R: ",R)
        # print("Constant: ", constant)
        force = np.array([0,0])
        force_change_dir = 0
        #print(neighbours)
        #print("Artificial Gravity Neighbours: ", neighbours)
        #print("Artificial Gravity Neighbours Magnitudes: ", (np.sqrt(np.square(neighbours[0])+np.square(neighbours[1]))))
        # print(R*1.7)
        loop_counter = 0
        
        for x,y in zip(positions[0],positions[1]):
            
            constant = 1/robot.lattice
                
            object_position = np.array([(neighbourhood[loop_counter].x-robot.x),(neighbourhood[loop_counter].y-robot.y)])
            
            #print("Object Distance: ", np.array([delta_x,delta_y]))
            #distance_0 = np.sqrt(np.square(delta_x)+np.square(delta_y))
            
            #neighbour = neighbours[i]
            #print(i)
            #storing current position 
            current_position = np.array([x,y])
            """if(not np.array_equal(current_position,object_position)): 
                print("Object Distance: ", object_position)
                print("Neighbourhood Distance: ", current_position)
            """
            #calculating magnitude of positions
            mag_unshifted = (magnitude(current_position[0],current_position[1]))
            """#print(mag_unshifted)
            if(mag_unshifted > R):
                #print("Larger Distance Value: ", mag_unshifted)
                if(mag_unshifted>1.7*R):
                    #print("Error! Distance is too large. Value is ",mag_unshifted)"""
            if(like):
                mag = mag_unshifted*constant
            else:
                mag = mag_unshifted

            
            #calculating the force
            numerator = (G*(robot.mass**2))
            denominator = mag**2

            
            #calling function to account for division by zero, if there is a zero denominator, then the force component with zero is returned as zero
            force_change = division_check(numerator,denominator)
            
            #calculating the unit vector of the position
            distance_unit = unit(current_position)
            
            #if the magnitude is bigger than R, then a force is added to draw the robots together             
            if(mag>R):
                force_change_dir = distance_unit
            
            #if the magnitude is smaller than R, then a force is added to push the robots apart 
            elif(mag<R):
                force_change_dir = -distance_unit
            else:
                force_change_dir = np.array([0,0])
       
            #calculating new force change based on direction and magnitude
            force_delta = force_change_dir*force_change

            #constraining force to the maximum
            if((force_delta[0]**2+force_delta[1]**2)>((max_force)**2)):
                #calculating unit vector of the force
                unit_force = unit([force_delta[0],force_delta[1]])
                
                #multiplying it by the maximum force
                updating_force = unit_force*max_force
                
                #setting the new force equal to the respective updated maximum force components
                force_delta[0] = updating_force[0]
                force_delta[1] = updating_force[1]
            force = force+force_delta
            loop_counter = loop_counter + 1
            
        #setting force maximum for sum of force contributions
        """if((force[0]**2+force[1]**2)>((max_force)**2)):
                #calculating unit vector of the force
                unit_force = unit([force[0],force[1]])
                
                #multiplying it by the maximum force
                updating_force = unit_force*max_force
                
                #setting the new force equal to the respective updated maximum force components
                force[0] = updating_force[0]
                force[1] = updating_force[1]"""
        #print("Finished")
        return force
    

In [49]:
#Function to calculate distance between a robot and the swarm
def dist(robot, x,y):
    #finding the distance between the current robot and the other robots
    x_dist = x-robot.x
    y_dist = y-robot.y
    return x_dist,y_dist

In [50]:
#Function to split swarm into different types
def split(swarm):
    type_0 = []
    type_1 = []
    for robot in swarm:
        if(robot.identifier == 0):
            type_0.append(robot)
        if(robot.identifier == 1):
            type_1.append(robot)

            
    return type_0,type_1
    

In [51]:
def broadcast(swarm, detection_list, set_R):
    #retrieving x and y positions
    x,y = positions(swarm)
   
    detecting = np.where(detection_list)
    list_detected = list(detecting[0])
    for i in list_detected:
        robot = swarm[i]
        x_copy,y_copy = x.copy(),y.copy()
        #finding the distance between the current robot and the other robots
        x_dist_unsorted = np.array(x_copy)-robot.x
        y_dist_unsorted = np.array(y_copy)-robot.y
        
        #calculating the distance from the robots to the current robot
        distance = np.sqrt(np.square(x_dist_unsorted)+np.square(y_dist_unsorted))
       
        #determining the robots within 1.5R distance
        distance_local = np.logical_and(distance>0,distance<multiply*R)   
        
        #creating the x and y distance matrices for robots within the local 1.5R distance
        x_dist = x_dist_unsorted[distance_local]
        y_dist = y_dist_unsorted[distance_local]
        
        neighbourhood = np.where(distance_local)
        list_neighbours = list(neighbourhood[0])

        neighbours = [swarm[i] for i in list_neighbours]
        #looping through the current robots neighbourhood
        for j in list_neighbours:
            swarm[j].R = set_R
    
            

In [52]:
#Function to calculate the G transition parameter
def G_square(max_force,R,power):
    '''
    Calculates the transition value between liquid and solid states for the G parameter

            Parameters:
                   max_force (float): the maximum float limit on the force
                   R (float): the chosen R seperation float value
                   power (float): the chosen power for the artificial gravity equation
                   
            Returns:
                   G_transition (float): the G transition value for the current swarm setup

    '''
    G_transition = (max_force*(R**power))/(2+2*math.sqrt(2))
    return G_transition

## Physics Constants

In [53]:
#setting physics constants
power = 2
R = 20
#max_force = 1
multiply = 1.5

In [54]:
#initialising parameters
conversion_ms = 1/3.6 #km/h to m/s conversion actor
timer = 0


#Robot Parameters
robot_number = 100
robot_speed = 2 #m/s
robot_radius = 1 
robot_mass = 1 #kgfro
x_robot = []
y_robot = []
min_neighbour = []
gen_random = 0 #the random angle

#dust devil parameters
dust_time = 170 #time in seconds the dust devil will survive 
dust_speed = 5 #m/s
dust_radius = 6
dust_devils = []
detection = 100

#Area Parameters
area = 2 #area in km^2 
side = 1 #each side length in m
per_square_km = 100/(60*60*24)
probability_dust = per_square_km*4#(per_square_km*area) #number of dust devils per second






In [90]:
lattice_constants = [math.sqrt(3),math.sqrt(2)]
G_scale = [0.9,0.95,1,1.05,1.1]
initialise_array = [20]
lattice = lattice_constants[0]
honeycomb = True
robot = 100#100
time = 50
#robot_speed_array = [1,2,3,4,5,6,7,8,9,10]
max_force_array = [1]
velocity_coefficient_array = [0.5]
original_path = "../Experiments/Testing_Graphing/"
try:
    os.mkdir(original_path)
except OSError:
    print ("Failure: Directory creation of %s failed" % original_path)
else:
    print ("Success: Directory creation of %s succeeded" % original_path)

values = []
timestep = 1
frequency = 1

R_array = [20]#[10,20,50,100,150,200,250]

grid = np.array([-500,500])
grid_length = 1000
grid_performance = []

for R_index in range(len(R_array)):
    velocity_coefficient = velocity_coefficient_array[0]
    max_force = max_force_array[0]
    R = R_array[R_index]
    G_state = G_transition(max_force,R,power)


    random_seed = random.randint(0,10000)
    random.seed(random_seed)

    performance_metric = []
    code = "R_" + str(R)
    path = (original_path + code + "/")
    try:
        os.mkdir(path)
    except OSError:
        print ("Failure: Directory creation of %s failed" % path)
    else:
        print ("Success: Directory creation of %s succeeded" % path)


    image = (original_path + "Images/")
    try:
        os.mkdir(image)
    except OSError:
        print ("Failure: Directory creation of %s failed" % image)
    else:
        print ("Success: Directory creation of %s succeeded" % image)

    #for initial in initial_array:

    #only set seed if eliminating bias
    #random_seed = os.urandom(8)
    #random.seed(random_seed)
    #setting up the robot setup
    swarm = initialise(robot, side, initialise_array[0],robot_speed,R,random_seed, lattice_constants)

    store_robots = np.zeros((2,robot,(time*timestep)))
    store_dust = np.zeros((2,robot,(time*timestep)))
    min_neighbours = []
    cluster_average = []
    store_dust = []
    dust_devils = []
    measurement_events = 0
    detection_metric = 0
    dust_count = 0 
    total_detection = []
    total_collision = []
    total_dust = []

    code = "G_Parameter_" + str(G_state)
    path = (path + code + "/")
    try:
        os.mkdir(path)
    except OSError:
        print ("Failure: Directory creation of %s failed" % path)
    else:
        print ("Success: Directory creation of %s succeeded" % path)
    for j in range(0,time*frequency):
        """if(j%100==0 and j>0):
            honeycomb = not honeycomb

        if(honeycomb):
            lattice = lattice_options[0]
        else:
            lattice = lattice_options[1]"""
        print(j)
        update_decay(swarm,R)
        #updating robots and storing robot/dust devil positions
        store_robots[:,:,j] = positions(swarm)
        store_dust.append(positions(dust_devils))

        #checking if 1 second has passed and updating velocities of the robots via the physics walk algorithm and updating status of dust devils
        if(j%frequency==0):
        #    dust_count = dust_count + dust(dust_devils,1,side,j,dust_speed,dust_time,timestep,frequency)
           physics_walk_honey(swarm,G_state,power,R,max_force,multiply,1,lattice,velocity_coefficient)    

        #calling update timestep method which updates positions of robots, and returns performance metrics
        measurement_events_change,detection_metric_change = update_timestep(swarm,dust_devils,timestep,frequency,min_neighbours,cluster_average,detection,R,multiply,20,5)
        measurement_events = measurement_events + measurement_events_change
        detection_metric = detection_metric + detection_metric_change

        #appending performance metrics into list for storage/post processing
        total_detection.append(detection_metric)
        total_collision.append(measurement_events)

        #updates dust devils positions
        update_dust(dust_devils)

        #storing the number of dust devils generated
        total_dust.append(dust_count)
    
    type_0_object,type_1_object = split(swarm)
    type_0 = positions(type_0_object)
    type_1 = positions(type_1_object)
    print(type_0)
    print(store_robots[:,0])
    current_grid_metric = grid_coverage(grid,store_robots[:,time*frequency-1],grid_length)
    #saving robot positions
    np.save(path + "Robots.npy",store_robots)

    #saving dust devil positions
    with open( path + "dust.txt", "w") as f:
        json.dump(store_dust, f)

    #saving performance metrics - these could not be stored in excel due to storage limits
    np.save(path + "Minimum Distance to Neighbours.npy",np.array(min_neighbours))
    np.save(path + "Cluster Average.npy", np.array(cluster_average))
    np.save(path + "Measurement Events Count.npy", np.array(total_collision))
    np.save(path + "Number of Dust Devils Detected.npy", np.array(total_detection))
    np.save(path + "Number of Dust Devils Generated.npy", np.array(total_dust))


    #adding values tested for a values excel spreadsheet for easy viewing
    values.append([len(swarm),timestep,G_state,power,R,multiply,max_force,robot_speed,min_neighbours[time*frequency-1],cluster_average[time*frequency-1],random_seed, G_state,measurement_events,detection_metric,dust_count])

    #the titles of the variables in the values array
    values_title = ['Number of Robots',
                            'Timestep',
                            'G', 
                            'Power', 
                            'Communication Range', 
                            'Multiplier',
                            'Max Force',
                            'Max Speed',
                            'Minimum Distance to Neighbours',
                            'Average Cluster Size',
                            'Seed',
                            'G Transition',
                            'Measurement Events Count',
                            'Number of Dust Devils Detected',
                            'Total Number of Dust Devils',
        ]

    #storing the constants used in the simulation
    data = {'Values':[len(swarm),timestep,G_state,power,R,multiply,max_force,robot_speed,random_seed, G_state]}
    titles =['Number of Robots',
                            'G',
                            'Timestep', 
                            'Power',
                            'Communication Range', 
                            'Multiplier',
                            'Max Force',
                            'Max Speed',
                            'Seed',
                            'G Transition',
        ]
    #setting the constants as a panda dataframe
    constants = pd.DataFrame(data, index = titles
                           ) 

    constants.to_excel(path + "Constants.xlsx")
    print("G_state: ",G_state)
    print("G_parameter: " ,G_state)
    #setting the start paths for the graphs and the tables
    graph_start_path = image + code + "- Graph_Beginning.png"
    table_start_path = image + "/" + code + "- Table_Beginning.png"

    #using the processing functions to create plotly graphs and tables for the figures in the first timestep
    graph_start = Processing_Functions_Tracking.graph_figure(store_robots,0,frequency,code)
    graph_start.write_image(graph_start_path)
    table_start = Processing_Functions_Tracking.table_figure(store_robots,0,frequency,constants,min_neighbours,cluster_average,total_collision,total_detection,total_dust)
    table_start.write_image(table_start_path)

    #combining the tables and the graphs using pillow
    Processing_Functions_Tracking.combine(graph_start_path,table_start_path)

    #setting the end paths for the graphs and the tables
    graph_end_path = image + code + "- Graph_End.png"
    table_end_path = image + "/" + code + "- Table_End.png"

    #using the processing functions to create plotly graphs and tables for the figures in the last timestep
    graph_end = Processing_Functions_Tracking.graph_figure(store_robots,time-1,frequency,code)
    graph_end.write_image(image + "/" + code + "- Graph_End.png")
    table_end = Processing_Functions_Tracking.table_figure(store_robots,time-1,frequency,constants,min_neighbours,cluster_average,total_collision,total_detection,total_dust)
    table_end.write_image(image + "/" + code + "- Table_End.png")
    print(type_0)
    #using the processing functions to create plotly graphs and tables for the figures in the last timestep
    graph_end_types = Processing_Functions_Tracking.graph_figure_split(type_0,type_1,frequency,code)
    graph_end_types.write_image(image + "/" + code + "- Graph_End_Types.png")
    table_end_types = Processing_Functions_Tracking.table_figure(store_robots,time-1,frequency,constants,min_neighbours,cluster_average,total_collision,total_detection,total_dust)
    table_end_types.write_image(image + "/" + code + "- Table_Types_End.png")
    
    #using the processing functions to create plotly graphs and tables for the figures in the last timestep
    graph_end_area_coverage = Processing_Functions_Tracking.graph_figure_area_coverage(type_0,type_1,frequency,code)
    graph_end_area_coverage.write_image(image + "/" + code + "- Graph_End_Area_Coverage.png")
    #table_end_types = Processing_Functions_Tracking.table_figure(store_robots,time-1,frequency,constants,min_neighbours,cluster_average,total_collision,total_detection,total_dust)
    #table_end.write_image(image + "/" + code + "- Table_Types_End.png")
    
    #combining the tables and the graphs using pillow
    Processing_Functions_Tracking.combine(graph_end_path,table_end_path)

    #plotting performance of the average of minimum neighbouring distance metric
    performance = Processing_Functions_Tracking.performance_graph(min_neighbours,np.linspace(0,len(min_neighbours),len(min_neighbours)*frequency,endpoint = False),frequency,code,"Time (s)","Minimum Average Neighbour Distance (m)")
    performance.write_image(image + "/" + code + "- Performance.png")

    #plotting performance of the dust devil measurement metric
    performance_intercept = Processing_Functions_Tracking.performance_graph(total_collision,np.linspace(0,len(total_collision),len(total_collision)*frequency,endpoint = False),frequency,code,"Time (s)","Count of Measurement Events")
    performance_intercept.write_image(image + "/" + code + "- Intercept Performance.png")

    #plotting performance of the cluster average of the swarm
    cluster = Processing_Functions_Tracking.performance_graph(cluster_average,np.linspace(0,len(cluster_average),len(cluster_average)*frequency,endpoint = False),frequency,code,"Time (s)","Average Cluster Size")
    cluster.write_image(image + "/" + code + "- Average Cluster Size.png")




    #setting panda dataframe for the values tested
    df = pd.DataFrame(values, columns = values_title)

    #saving at as an excel spreadsheet
    df.to_excel(path + "Results.xlsx")


    #performance_metric_np = np.array(performance_metric)

    #saving performance metrics
    #np.save(original_path + "Performance Metric " + str(timestep) + ".npy", performance_metric_np)

    #plotting performance of the different timesteps
    #performance_overall = Processing_Functions_BP.performance_graph(performance_metric_np[:,1],performance_metric_np[:,0],1,code,"Number of Timesteps in 1 Second","Number of Dust Devils Detected")
    #performance_overall.write_image(new_path + "Overall Performance.png")
    grid_performance.append([current_grid_metric,R])

"""
    performance_metric.append([robot,total_detection[-1]])
"""

performance_metric_np = np.array(grid_performance)
np.save(original_path + "Grid Performance Metric.npy", performance_metric_np)
performance_overall = Processing_Functions_Tracking.performance_graph(performance_metric_np[:,0],performance_metric_np[:,1],1,code,"Desired Seperation Distance, R (m) ","Grid Coverage Percentage")
performance_overall.write_image(original_path + "Overall Performance.png")


Success: Directory creation of ../Experiments/Testing_Graphing/ succeeded
Success: Directory creation of ../Experiments/Testing_Graphing/R_20/ succeeded
Success: Directory creation of ../Experiments/Testing_Graphing/Images/ succeeded
Initialising..
[5.214442293951629, -42.2645018383244, 22.224983062631317, -24.711912252323316, -5.1822991713000475, -15.925239559816308, -47.95052168321164, -8.024089930344369, 4.870554263668499, 9.853212234287204, -9.883038598192615, 19.083180491549232, 5.452187916874306, 14.44973241171381, -22.863917772226998, -49.386624406985995, 3.2080007358309133, 10.734695575428134, 8.838194263704661, -22.604268487297844, -18.482005843525762, -46.70475454913175, -37.81201086816948, 46.13564172180665, 12.489023643822888, -17.566099668601012, -36.63356529734965, -28.342992373319138, -28.337275671592476, -6.603509137537076, 15.947332742141068, -4.298837224814974, 5.047740195319939, -19.949133749024533, 9.347828743591679, -8.901432023145322, -10.940582753563788, -14.3101

G_Parameter_115.47005383792516
G_Parameter_115.47005383792516
G_Parameter_115.47005383792516
G_Parameter_115.47005383792516


In [121]:
from matplotlib.pyplot import plot
type_0,type_1 = split(swarm)
x_0,y_0 = np.array(positions(type_0))
x_1,y_1 = np.array(positions(type_1))
plot.grid()
plot(x_0, y_0, 'bo')
plot(x_1, y_1, 'go')

AttributeError: 'function' object has no attribute 'grid'

In [176]:
from matplotlib.pyplot import plot
type_0,type_1 = split(swarm)
x_0,y_0 = np.array(positions(type_0))
x_1,y_1 = np.array(positions(type_1))
x_overall = np.concatenate((x_0, x_1), axis=None)
y_overall = np.concatenate((y_0,y_1),axis = None)
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=x_0,
    y=y_0,
    mode='markers',
    showlegend=False,
    marker=dict(
        symbol='circle',
        opacity=1,
        color='red',
        size=8,
        line=dict(width=1),
    )
))
fig.add_trace(go.Scatter(
    x=x_1,
    y=y_1,
    mode='markers',
    showlegend=False,
    marker=dict(
        symbol='circle',
        opacity=1,
        color='blue',
        size=8,
        line=dict(width=1),
    )
))

fig.add_trace((go.Histogram2d(x=x_overall, y=y_overall,
        autobinx=False,
        xbins=dict(start=-200, end=200, size=20),
        autobiny=False,
        ybins=dict(start=-200, end=200, size=20),
        zmax=1,            
        zauto=False,
        showscale = False,
        colorscale=["white", "black"]
        )))
fig.update_xaxes(showgrid=True,showline=True, linewidth=2, linecolor='black',mirror=True)
fig.update_yaxes(showgrid=True,showline=True, linewidth=2, linecolor='black',mirror=True)
fig.update_xaxes(showgrid=True,zeroline=False, gridwidth=1, gridcolor='black')
fig.update_yaxes(showgrid=True,zeroline=False, gridwidth=1, gridcolor='black')
fig.update_traces(opacity=0.4, selector=dict(type='histogram2d'))
fig.update_layout(
#     height=900,width=1150,
     xaxis = dict(
         tickmode = 'linear',
         tick0 = 0,
         dtick = 200/10
     ),
                       yaxis = dict(
         tickmode = 'linear',
         tick0 = 0,
         dtick = 200/10
     ))
fig.show()

In [ ]:
"""original_path = "../Experiments/Initial_Types_Test/"
try:
    os.mkdir(original_path)
except OSError:
    print ("Failure: Directory creation of %s failed" % original_path)
else:
    print ("Success: Directory creation of %s succeeded" % original_path)
    
code = "Run_" + str(initialise_array[i])
path = (original_path + code + "/")
try:
        os.mkdir(path)
except OSError:
    print ("Failure: Directory creation of %s failed" % path)
else:
    print ("Success: Directory creation of %s succeeded" % path)


image = (original_path + "Images/")
try:
    os.mkdir(image)
except OSError:
    print ("Failure: Directory creation of %s failed" % image)
else:
    print ("Success: Directory creation of %s succeeded" % image)

#saving performance metrics - these could not be stored in excel due to storage limits
np.save(path + "Minimum Distance to Neighbours.npy",np.array(min_neighbours))
np.save(path + "Cluster Average.npy", np.array(cluster_average))
np.save(path + "Measurement Events Count.npy", np.array(total_collision))
np.save(path + "Number of Dust Devils Detected.npy", np.array(total_detection))
np.save(path + "Number of Dust Devils Generated.npy", np.array(total_dust))


#adding values tested for a values excel spreadsheet for easy viewing
values.append([len(swarm),timestep,G_state,power,R,multiply,max_force,robot_speed,min_neighbours[time*frequency-1],cluster_average[time*frequency-1],random_seed, G_state,measurement_events,detection_metric,dust_count])

#the titles of the variables in the values array
values_title = ['Number of Robots',
                        'Timestep',
                        'G', 
                        'Power', 
                        'Communication Range', 
                        'Multiplier',
                        'Max Force',
                        'Max Speed',
                        'Minimum Distance to Neighbours',
                        'Average Cluster Size',
                        'Seed',
                        'G Transition',
                        'Measurement Events Count',
                        'Number of Dust Devils Detected',
                        'Total Number of Dust Devils',
    ]

#storing the constants used in the simulation
data = {'Values':[len(swarm),timestep,G_state,power,R,multiply,max_force,robot_speed,random_seed, G_state]}
titles =['Number of Robots',
                        'G',
                        'Timestep', 
                        'Power',
                        'Communication Range', 
                        'Multiplier',
                        'Max Force',
                        'Max Speed',
                        'Seed',
                        'G Transition',
    ]
#setting the constants as a panda dataframe
constants = pd.DataFrame(data, index = titles
                       ) 

constants.to_excel(path + "Constants.xlsx")

#setting the start paths for the graphs and the tables
graph_start_path = image + code + "- Graph_Beginning.png"
table_start_path = image + "/" + code + "- Table_Beginning.png"

#using the processing functions to create plotly graphs and tables for the figures in the first timestep
graph_start = Processing_Functions_Tracking.graph_figure(store_robots,0,frequency,code)
graph_start.write_image(graph_start_path)
table_start = Processing_Functions_Tracking.table_figure(store_robots,0,frequency,constants,min_neighbours,cluster_average,total_collision,total_detection,total_dust)
table_start.write_image(table_start_path)

#combining the tables and the graphs using pillow
Processing_Functions_Tracking.combine(graph_start_path,table_start_path)

#setting the end paths for the graphs and the tables
graph_end_path = image + code + "- Graph_End.png"
table_end_path = image + "/" + code + "- Table_End.png"

#using the processing functions to create plotly graphs and tables for the figures in the last timestep
graph_end = Processing_Functions_Tracking.graph_figure(store_robots,time-1,frequency,code)
graph_end.write_image(image + "/" + code + "- Graph_End.png")
table_end = Processing_Functions_Tracking.table_figure(store_robots,time-1,frequency,constants,min_neighbours,cluster_average,total_collision,total_detection,total_dust)
table_end.write_image(image + "/" + code + "- Table_End.png")

#combining the tables and the graphs using pillow
Processing_Functions_Tracking.combine(graph_end_path,table_end_path)

#plotting performance of the average of minimum neighbouring distance metric
performance = Processing_Functions_Tracking.performance_graph(min_neighbours,np.linspace(0,len(min_neighbours),len(min_neighbours)*frequency,endpoint = False),frequency,code,"Time (s)","Minimum Average Neighbour Distance (m)")
performance.write_image(image + "/" + code + "- Performance.png")

#plotting performance of the dust devil measurement metric
performance_intercept = Processing_Functions_Tracking.performance_graph(total_collision,np.linspace(0,len(total_collision),len(total_collision)*frequency,endpoint = False),frequency,code,"Time (s)","Count of Measurement Events")
performance_intercept.write_image(image + "/" + code + "- Intercept Performance.png")

#plotting performance of the cluster average of the swarm
cluster = Processing_Functions_Tracking.performance_graph(cluster_average,np.linspace(0,len(cluster_average),len(cluster_average)*frequency,endpoint = False),frequency,code,"Time (s)","Average Cluster Size")
cluster.write_image(image + "/" + code + "- Average Cluster Size.png")




#setting panda dataframe for the values tested
df = pd.DataFrame(values, columns = values_title)

#saving at as an excel spreadsheet
df.to_excel(path + "Results.xlsx")


#performance_metric_np = np.array(performance_metric)

#saving performance metrics
#np.save(original_path + "Performance Metric " + str(timestep) + ".npy", performance_metric_np)

#plotting performance of the different timesteps
#performance_overall = Processing_Functions_BP.performance_graph(performance_metric_np[:,1],performance_metric_np[:,0],1,code,"Number of Timesteps in 1 Second","Number of Dust Devils Detected")
#performance_overall.write_image(new_path + "Overall Performance.png")
"""

In [ ]:
path =  "Experiments/Types_Testing/Run_1/"
store_robots = np.load(path + "Robots.npy")

with open(path + "dust.txt", "r") as f:
     dust = json.load(f)

constants = pd.read_excel(path + "Constants.xlsx", index_col=0)

In [ ]:
#the update layout function
def update_layout():
    #contains the graph, the interval timing and the live updating table
    app.layout = html.Div(children=[html.Div(
    [
    dcc.Graph(id = 'live_graph'),
    dcc.Interval(
        id = 'graph-update',
        interval = 100, #milliseconds
        n_intervals = 0
        ),
    ],style = {'display': 'inline-block'} ),
        html.Div(
            [
                dcc.Graph(id = 'live_table'),
                dcc.Interval(
                id = 'variable-update',
                interval = 100, #milliseconds
                n_intervals = 0
                )    
                 ],
        style = {'display': 'inline-block','vertical-align': 'top' }    
    )],style={'width': '100%', 'display': 'inline-block'})

update_layout()

In [ ]:
#callback is the updating process
@app.callback(Output('live_graph','figure'),
              [Input('graph-update', 'n_intervals')])

#the actual update for the graph
def update_graph(n):
    #initialising global variables
    global timer
    

    position_robot = store_robots[:,:,timer]
    position_dust = store_dust[timer]
    #creating scatter plot of robots
    data = go.Scatter(
        x=list(position_robot[0]),
        y=list(position_robot[1]),
        name = 'Robots',
        mode = 'markers',
    )
    
    #creating the plotly figure with the robot data
    fig = go.Figure(
        { "data": data, "layout": go.Layout(yaxis=dict(range=[-1000, 1000]),xaxis = dict(range=[-1000,1000]))
        }
    )

    fig.add_trace(go.Scatter(
          x= position_dust[0],
          y= position_dust[1],
          name='Dust Devil',
          mode = 'markers',
          marker_color='rgba(255, 182, 193, .9)',
      ))
    
    #updating layout with circles and different formatting'''
    fig.update_layout(title="<b>Physics Based Swarm Experiment </b>",
    title_x=0.5,
    xaxis_title="X Position (m)",
    yaxis_title="Y Position (m)",
    margin=dict(
        t=50, # top margin: 30px, you want to leave around 30 pixels to
              # display the modebar above the graph.
         # bottom margin: 10px
        l=10, # left margin: 10px
        r=10, # right margin: 10px
    ),
    height=900,width=1150,
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 50
    ),
                      yaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 50
    )
                     )
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black',mirror=True)
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black',mirror=True)
    #incrementing timer
    timer = timer + 1
    
    #updating the layout
    update_layout()
    return fig

#app callback for the table
@app.callback(Output('live_table','figure'),
              [Input('variable-update', 'n_intervals')])

def update_variables(n):
    fig = Processing_Functions_Tracking.table_figure(store_robots,timer,timestep,constants,min_neighbours,cluster_average,total_collision,total_detection,total_dust)
                
    fig.update_layout(width = 650, height = 800,margin=dict(
    t=200, # top margin: 30px, you want to leave around 30 pixels to
              # display the modebar above the graph.
        b=10, # bottom margin: 10px
        l=10, # left margin: 10px
        r=10,
    ))
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
    return fig

In [ ]:
#runs by default
if __name__ == '__main__':
    #running the app server for the simulator
    app.run_server(debug=False, host = '127.0.0.2')

In [ ]:
def R_return(swarm):
    R_array = []
    for i in swarm:
        R_array.append(i.R)
    return R_array

In [ ]:
print(R_return(swarm))
print(positions(swarm))

In [ ]:
R=20
x_0 =np.array([  0,  -6,   9,   0, -10,   5,  -5, -13, -10,   0, -25,  10,  -1,
         4,  -5,   6, -21, -24, -18,   2, -19,  -9, -11,  -4,  10,   1,
         3, -14,   2, -11, -18, -12,   6,  -7, -20, -27, -20,  -6, -13,
        -9,  12,   7, -21,   8, -14, -11, -14,   8,  11, -14])
y_0 = np.array([ 0, 10,  3,  2, 19, 27, -3, 25, 29, 35, 31, 17, 26, 35, 21, 32, 29,
       36, 34, -4, 19,  7, 17, -3, -4, -2, 30, 25, 17, 15, 33, 21, 34,  0,
        1, 14, 36, 14, 20, 19, 21, 22, 21, -1, -3, 10, 35,  5,  6, 18])

#calculating the distance from the robots to the current robot, split up according to robot type
distance_0 = np.sqrt(np.square(x_0)+np.square(y_0))

print(distance_0)

#finding the corresponding booleans
multiply_like = 1.3
multiply_unlike = 1.7
constant = 0

distance_local_0 = np.logical_and(distance_0>0,distance_0<multiply_like*R)

print(distance_local_0)
print(distance_0[distance_local_0])




x_dist_0 = x_0[distance_local_0]
y_dist_0 = y_0[distance_local_0]
print("X Positions in Neighbourhood: ", x_dist_0)
print("Y Positions in Neighbourhood: ", y_dist_0)
positions_0 = np.array([x_dist_0,y_dist_0])
print(np.sqrt(np.square(positions_0[0])+np.square(positions_0[1])))
for x,y in zip(positions_0[0],positions_0[1]):
    print(magnitude(x,y))
    current_position = np.array([x,y])


    #calculating magnitude of positions
    mag_unshifted = (magnitude(current_position[0],current_position[1]))
    print(mag_unshifted)

In [ ]:
#function to calculate area coverage using grid metric
def grid_coverage(grid,positions,length):
    #creating the grid bins for area coverage
    x_bins = np.linspace(grid[0],grid[1],length+1)
    y_bins = np.linspace(grid[0],grid[1],length+1)
    
    x_positions = positions[0]
    y_positions = positions[1]
    #initialising empty coordinates array for use in the grid metric
    coordinates = []
    for i in range(len(x_positions)):
        #setting current position
        x_current = x_positions[i]
        y_current = y_positions[i]
        #searching for the x anf y coordinates using the numpy search sorted function combined with the previously defined bins
        x = np.searchsorted(x_bins, x_current)
        y = np.searchsorted(y_bins, y_current)
        if((0<=x<length+1) and (0<=y<length+1)):
            coordinates.append((x,y))
        print(coordinates)


    rm_duplicates = list(set(coordinates)) #set is used to remove duplicates, as sets require distinct elements
    area_coverage = len(rm_duplicates)*0.01 #converting it to a percentage
    total = length*length
    return area_coverage

In [ ]:
x_0 =np.array([  0,  -6,   9,   0, -10,   5,  -5, -13, -10,   0, -25,  10,  -1,
         4,  -5,   6, -21, -24, -18,   2, -19,  -9, -11,  -4,  10,   1,
         3, -14,   2, -11, -18, -12,   6,  -7, -20, -27, -20,  -6, -13,
        -9,  12,   7, -21,   8, -14, -11, -14,   8,  11, -14])
y_0 = np.array([ 0, 10,  3,  2, 19, 27, -3, 25, 29, 35, 31, 17, 26, 35, 21, 32, 29,
       36, 34, -4, 19,  7, 17, -3, -4, -2, 30, 25, 17, 15, 33, 21, 34,  0,
        1, 14, 36, 14, 20, 19, 21, 22, 21, -1, -3, 10, 35,  5,  6, 18])
(grid_coverage(np.array([-500,500]),np.array([x_0,y_0]),100))

In [ ]:
grid = [-500,500]
x = np.linspace(-500,500,11)
y = np.linspace(-500,500,11)
print(x)
print(y)

position_array = np.array([[-6000,6000],[-6000,6000]]) #store_robots[:,:,1999]
print("Positions:",position_array)
print("Independent review: ",position_array[0])
coordinates = []
for i in range(len(position_array[0])):
    position = (position_array[:,i])
    coordinates.append(((np.searchsorted(x, position[0])),(np.searchsorted(y, position[1]))))
    print(coordinates)


rm_duplicates = list(set(coordinates))
print(rm_duplicates)
print(len(coordinates))
print(len(rm_duplicates))

In [65]:

x_0 =np.array([  0,  -6,   9,   0, -10,   5,  -5, -13, -10,   0, -25,  10,  -1,
         4,  -5,   6, -21, -24, -18,   2, -19,  -9, -11,  -4,  10,   1,
         3, -14,   2, -11, -18, -12,   6,  -7, -20, -27, -20,  -6, -13,
        -9,  12,   7, -21,   8, -14, -11, -14,   8,  11, -14])
y_0 = np.array([ 0, 10,  3,  2, 19, 27, -3, 25, 29, 35, 31, 17, 26, 35, 21, 32, 29,
       36, 34, -4, 19,  7, 17, -3, -4, -2, 30, 25, 17, 15, 33, 21, 34,  0,
        1, 14, 36, 14, 20, 19, 21, 22, 21, -1, -3, 10, 35,  5,  6, 18])
fig = go.Figure(go.Histogram2d(x=x_0, y=y_0,
        autobinx=False,
        xbins=dict(start=-500, end=500, size=50),
        autobiny=False,
        ybins=dict(start=-500, end=500, size=50),
        zmax=1,            
        zauto=False,
        showscale = False
        ))
fig.add_trace(go.Scatter(
    x=x_0,
    y=y_0,
    mode='markers',
    showlegend=False,
    marker=dict(
        symbol='circle',
        opacity=0.7,
        color='white',
        size=8,
        line=dict(width=1),
    )
))
fig.show()